In [1]:
from thesis_initialise import *

import numpy as _np

In [251]:
from dataclasses import dataclass, fields, Field, InitVar

In [346]:
################################################################################

from functools import lru_cache as _lru_cache, reduce as _reduce
import numpy as _np

@dataclass
class ChesterWorld:
    nTimes: int = 24
    levels: int = 3
    popDensity: int = 1500 # persons per km 2
    res: float = 0.016 # innermost edge km
    init: InitVar(bool) = True
    def __post_init__(self, init):
        self.maxLevel = (levels := self.levels) - 1
        self.nTiles = nTiles = 2 ** (8 * levels)
        self.length = length = (res := self.res) * 16 ** levels
        self.area = area = length ** 2
        self.nAgents = nAgents = round(self.popDensity * area)
        self.agentsPerTile = nAgents / nTiles
        self.agentIDs = _np.arange(nAgents)
        self._hashVal = np.random.randint(10 ** 12, 10 ** 13 - 1)
        if init:
            self.initialise()
    def initialise(self):
        shape = (self.nAgents, self.nTimes, self.levels)
        self.schedule = _np.random.randint(0, 2**8, shape, 'uint8')
#         self.places = _np.zeros(self.nAgents,)
    @_lru_cache
    def get_potential_contacts(self, agentID: int, level: int = None, /):
        level = self.maxLevel if level is None else level
        if level:
            shortlist = self.get_potential_contacts(agentID, level - 1)
        else:
            shortlist = self.agentIDs
        schedule = self.schedule[:, :, level][shortlist]
        matches = schedule == schedule[agentID]
        return shortlist[_np.unique(_np.nonzero(matches)[0])]
    def __hash__(self):
        return self._hashVal

################################################################################

#         if level = 0:
#             schedule = self.schedule
#         else:
#             schedule = self.get_potential_contacts(agentID, level - 1)
#         schedule = self.schedule[:, :, :level+1]
#         contacts = _np.nonzero(schedule == schedule[agentID])[0]
#         return sorted(set(contacts))
#     def get_contacts(self, agentID: int):
#         schedule = self.schedule
#         f
#         return _reduce(
#             self.get_potential_contacts,
#             (agentID, )
#             )

In [347]:
%time model = ChesterWorld(nTimes = 1)
%time contacts = model.get_potential_contacts(0, 0)
%time contacts = model.get_potential_contacts(0, 1)
%time contacts = model.get_potential_contacts(0, 2)
print(f"nContacts = {len(contacts)}")
assert all(_np.any(model.schedule[0] == model.schedule[contact]) for contact in contacts)
assert all(_np.any(model.schedule[0, :, 0] == model.schedule[contact, :, 0]) for contact in contacts)

CPU times: user 40.9 ms, sys: 111 ms, total: 152 ms
Wall time: 155 ms
CPU times: user 46.7 ms, sys: 0 ns, total: 46.7 ms
Wall time: 46.3 ms
CPU times: user 790 µs, sys: 218 µs, total: 1.01 ms
Wall time: 1.02 ms
CPU times: user 46 µs, sys: 24 µs, total: 70 µs
Wall time: 75.6 µs
nContacts = 1


In [342]:
%time model = ChesterWorld(nTimes = 24)
%time contacts = model.get_potential_contacts(0, 0)
%time contacts = model.get_potential_contacts(0, 1)
%time contacts = model.get_potential_contacts(0, 2)
print(f"nContacts = {len(contacts)}")
assert all(_np.any(model.schedule[0] == model.schedule[contact]) for contact in contacts)
assert all(_np.any(model.schedule[0, :, 0] == model.schedule[contact, :, 0]) for contact in contacts)

CPU times: user 694 ms, sys: 116 ms, total: 810 ms
Wall time: 814 ms
CPU times: user 713 ms, sys: 865 ms, total: 1.58 s
Wall time: 1.59 s
CPU times: user 95.5 ms, sys: 0 ns, total: 95.5 ms
Wall time: 95.5 ms
CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 11.8 ms
nContacts = 4697


In [348]:
self = ChesterWorld(nTimes = 4)

In [355]:
contacts = self.get_potential_contacts(0)

In [361]:
subSchedule = (schedule := self.schedule)[:, :, -1]
contactMatrix = schedule[_np.nonzero(subSchedule[contacts] == subSchedule[0])]
print(self.schedule[0])
print(contactMatrix)

[[171 125 214]
 [  3 238 247]
 [153 218   2]
 [ 90  44  81]]
[[171 125 214]
 [  3 238 247]
 [153 218   2]
 [ 90  44  81]
 [193 202  26]
 [231 193 147]
 [154 253 178]
 [ 87 117  20]
 [ 90 232 224]
 [160 194  13]
 [ 72  40  18]
 [ 28 154   9]
 [ 37 213  87]
 [ 33 156  16]
 [167 139  82]
 [ 61 201 202]
 [156  39 254]
 [238 127 235]
 [165  76 206]
 [126 244  47]
 [136 176 194]
 [ 20 195 171]
 [ 47  24 170]
 [ 99  86 147]
 [167 224 213]
 [151  83  41]
 [226  17 111]
 [147 141  55]
 [158 108 161]
 [ 31  87  93]
 [138 180  29]]


In [368]:
contacts

array([      0,   93882,  405289,  893499,  956509, 1211134, 1348901,
       2083650, 2248550, 2708725, 2897214, 3214244, 3223969, 3236468,
       3268390, 3424756, 3484654, 3876479, 3886267, 3948473, 4915086,
       4948541, 5168450, 5197619, 5457133, 5717441, 5748002])

In [365]:
tracerSchedule = schedule[0]

In [367]:
tracerSchedule

array([[171, 125, 214],
       [  3, 238, 247],
       [153, 218,   2],
       [ 90,  44,  81]], dtype=uint8)

In [364]:
subSchedule[contacts] == subSchedule[0]

array([[ True,  True,  True,  True],
       [False,  True, False, False],
       [ True, False, False, False],
       [False, False,  True, False],
       [False,  True, False, False],
       [False, False,  True, False],
       [False,  True, False, False],
       [False,  True, False, False],
       [ True,  True, False, False],
       [False,  True, False, False],
       [False, False, False,  True],
       [False, False, False,  True],
       [False,  True, False, False],
       [False, False,  True, False],
       [False,  True, False, False],
       [ True, False, False, False],
       [False, False,  True, False],
       [ True, False, False, False],
       [False, False, False,  True],
       [ True, False, False, False],
       [False,  True, False, False],
       [ True, False, False, False],
       [False, False,  True, False],
       [False,  True, False, False],
       [False, False, False,  True],
       [False,  True, False, False],
       [False, False, False,  True]])

In [358]:
_np.argwhere(contactMatrix)

array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 1,  1],
       [ 2,  0],
       [ 3,  2],
       [ 4,  1],
       [ 5,  2],
       [ 6,  1],
       [ 7,  1],
       [ 8,  0],
       [ 8,  1],
       [ 9,  1],
       [10,  3],
       [11,  3],
       [12,  1],
       [13,  2],
       [14,  1],
       [15,  0],
       [16,  2],
       [17,  0],
       [18,  3],
       [19,  0],
       [20,  1],
       [21,  0],
       [22,  2],
       [23,  1],
       [24,  3],
       [25,  1],
       [26,  3]])

array([[[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]],

       [[False, False,  True],
        [False, False, False],
        [ True, False, False],
        [False,  True, False]],

       [[False, False, False],
        [ True,  True, False],
        [False, False,  True],
        [False, False, False]],

       [[ True,  True, False],
        [False, False, False],
        [False, False, False],
        [False, False,  True]],

       [[False,  True, False],
        [False, False, False],
        [False, False, False],
        [ True, False,  True]],

       [[False,  True, False],
        [ True, False, False],
        [False, False, False],
        [False, False,  True]],

       [[False, False,  True],
        [False, False, False],
        [False,  True, False],
        [ True, False, False]],

       [[False, False, False],
        [ True,  True,  True],
        [False, False, False],
        [False, False, Fa